In [8]:
!pip install IProgress

In [47]:
import copy
import datasets
from transformers import LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained("/home/zhiyuan/llama2-recipe/Llama-2-13b-chat-hf")
split = 'test'

In [48]:
dataset = datasets.load_dataset("json", data_files='CommSense.json', field=split, split='train')

Using custom data configuration default-66edbc18c775175d
Reusing dataset json (/home/zhiyuan/.cache/huggingface/datasets/json/default-66edbc18c775175d/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


In [49]:
print(dataset)

Dataset({
    features: ['question', 'answer'],
    num_rows: 15
})


In [50]:
def apply_prompt_template(sample):
    return {
        "question": sample["question"],
        "answer": sample["answer"],
    }
print(dataset)

Dataset({
    features: ['question', 'answer'],
    num_rows: 15
})


In [51]:
dataset = dataset.map(apply_prompt_template, remove_columns=list(dataset.features))

  0%|          | 0/15 [00:00<?, ?ex/s]

In [52]:

def tokenize_add_label(sample):
    question = tokenizer.encode(tokenizer.bos_token + sample["question"], add_special_tokens=False)
    answer = tokenizer.encode(sample["answer"] +  tokenizer.eos_token, add_special_tokens=False)

    sample = {
        "input_ids": question + answer,
        "attention_mask" : [1] * (len(question) + len(answer)),
        "labels": [-100] * len(question) + answer,
    }

    return sample
    
dataset = dataset.map(tokenize_add_label, remove_columns=list(dataset.features))

  0%|          | 0/15 [00:00<?, ?ex/s]

In [53]:
dataset_val = dataset
print(dataset_val)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 15
})


In [20]:
from torch.utils.data import Dataset
class ConcatDataset(Dataset):
    def __init__(self, dataset, chunk_size=4096):
        self.dataset = dataset
        self.chunk_size = chunk_size

        self.samples = []

        buffer = {
            "input_ids": [],
            "attention_mask": [],
            "labels": [],
            }

        for sample in tqdm(self.dataset, desc="Preprocessing dataset", dynamic_ncols=True):
            buffer = {k: v + sample[k] for k,v in buffer.items()}

            while len(next(iter(buffer.values()))) > self.chunk_size:
                self.samples.append({k: v[:self.chunk_size] for k,v in buffer.items()})
                buffer = {k: v[self.chunk_size:] for k,v in buffer.items()}

    def __getitem__(self, idx):
        return self.samples[idx]

    def __len__(self):
        return len(self.samples)